In [3]:
from keras.models import Model
from keras.layers import Conv2D, Embedding, MaxPooling2D, Input, Dense, Dot
from keras.layers import Flatten, Reshape, Activation ,Concatenate, Reshape, Dropout
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.layers.wrappers import TimeDistributed, Bidirectional

from keras import optimizers
import tensorflow as tf
from keras.layers.recurrent import LSTM

import numpy as np

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# input = (?, length, frequency) <- librosa mel-spectrum

In [84]:
class LSTM_attention():
    def __init__(self, num_hidden=100):
        self.num_hidden = num_hidden
        
    def LSTM(self, len_div, num_freq):
        self.LSTM_FOR = LSTM(self.num_hidden, return_sequences=False, go_backwards=False, name='LSTM_for_shared') 
        self.LSTM_INV = LSTM(self.num_hidden, return_sequences=False, go_backwards=True, name='LSTM_inv_shared')
        
        inputs = Input(shape=(len_div, num_freq), name='input')

        lstm_for = self.LSTM_FOR(inputs)
        lstm_inv =self.LSTM_INV(inputs)
        
        conc = Concatenate(axis=-1, name='concat')([lstm_for, lstm_inv])

        drop = Dropout(rate=0.05, name='drop')(conc)
        dens = Dense(80, name='dense')(drop)
        pred = Activation('softmax',name='pred')(dens)

        model = Model(inputs=inputs, outputs=pred)
        return model

    def LSTM2(self, len_div, num_freq):
        LSTM_FOR = self.LSTM_FOR
        LSTM_INV = self.LSTM_INV
        
        LSTM_FOR.return_sequences = True
        LSTM_INV.return_sequences = True
        
        inputs = Input(shape=(len_div, num_freq), name='input')

        lstm_for = LSTM_FOR(inputs)
        lstm_inv = LSTM_INV(inputs)
        
        conc = Concatenate(axis=-1, name='concat')([lstm_for, lstm_inv])

        drop = Dropout(rate=0.05, name='drop')(conc)
        dens = Dense(80, name='dense')(drop)
        pred = Activation('softmax',name='pred')(dens)

        model = Model(inputs=inputs, outputs=pred)
        return model

In [85]:
lstm_attn = LSTM_attention()
model = lstm_attn.LSTM(len_div=None, num_freq=128)
model2 = lstm_attn.LSTM2(len_div=512, num_freq=128)

In [86]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, None, 128)    0                                            
__________________________________________________________________________________________________
LSTM_for_shared (LSTM)          multiple             91600       input[0][0]                      
__________________________________________________________________________________________________
LSTM_inv_shared (LSTM)          multiple             91600       input[0][0]                      
__________________________________________________________________________________________________
concat (Concatenate)            (None, 200)          0           LSTM_for_shared[0][0]            
                                                                 LSTM_inv_shared[0][0]            
__________

In [87]:
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 512, 128)     0                                            
__________________________________________________________________________________________________
LSTM_for_shared (LSTM)          multiple             91600       input[0][0]                      
__________________________________________________________________________________________________
LSTM_inv_shared (LSTM)          multiple             91600       input[0][0]                      
__________________________________________________________________________________________________
concat (Concatenate)            (None, 512, 200)     0           LSTM_for_shared[1][0]            
                                                                 LSTM_inv_shared[1][0]            
__________